데이터 수집
freesound


In [ ]:
wav = '경로'
(file_dir, file_id) = os.path.split(wav)
print("file_dir:", file_dir)
print("file_id:", file_id)


In [ ]:
# 음성 파일을 librosa로 불러올 것이다.
# librosa는 별도로 sr(sampling rate)를 설정하지 않으면 default sr이 22500으로 되어있다.
y, sr = librosa.load(wav, sr=16000)
time = np.linspace(0, len(y)/sr, len(y)) # time axis

fig, ax1 = plt.subplots() # plot
ax1.plot(time, y, color = 'b', label='speech waveform')
ax1.set_ylabel("Amplitude") # y 축
ax1.set_xlabel("Time [s]") # x 축
plt.title(file_id) # 제목
plt.savefig(file_id+'.png')
plt.show()

#우선 아래의 코드를 통해 음성 신호를 절반만을 가져오고, 절반을 그려보도록 하자.
half = len(y)/2
y2 = y[:round(half)]
time2 = np.linspace(0, len(y2)/sr, len(y2))
fig2, ax2 = plt.subplots()
ax2.plot(time2, y2, color = 'b', label='speech waveform')
ax1.set_ylabel("Amplitude") # y 축
ax1.set_xlabel("Time [s]") # x 축
plt.title('cut '+file_id)
plt.savefig('cut_half '+file_id+'.png')
plt.show()

#librosa를 이용한 저장
librosa.output.write_wav('cut_file.wav', y2, sr)


### 음성 데이터 Resampling
resampling 되고나서 librosa를 이용하여 그대로 저장을 하고 윈도우에서 그 음성을 다운받아서 확인해볼 때 읽히지 않을때가 있다.
librosa 로 resample 이후에 soundfile의 write를 사용하여 저장하면 이 문제를 해결

In [ ]:
# librosa 라이브러리를 이용한 downsampling
def down_sample(input_wav, origin_sr, resample_sr):
    y, sr = librosa.load(input_wav, sr=origin_sr)
    resample = librosa.resample(y, sr, resample_sr)
    print("original wav sr: {}, original wav shape: {}, resample wav sr: {}, resmaple shape: {}".format(origin_sr, y.shape, resample_sr, resample.shape))
    plt.figure(figsize=(10, 4))
    plt.subplot(2, 1, 1)
    time1 = np.linspace(0, len(y) / sr, len(y))
    plt.plot(time1, y)
    plt.title('Original Wav')

    plt.subplot(2, 1, 2)
    time2 = np.linspace(0, len(resample) / resample_sr, len(resample))
    plt.plot(time2, resample)
    plt.title('Resampled Wav')

    plt.tight_layout()
    plt.savefig('compare_16k_vs_8k.png')
 # 음성 데이터 저장
sf.write('./' + '16k.wav' , y, origin_sr, format='WAV', endian='LITTLE', subtype='PCM_16')
sf.write('./' + '8k.wav', resample, resample_sr, format='WAV', endian='LITTLE', subtype='PCM_16')

man_original_data = '/mnt/junewoo/workspace/transform/test_wav/man_wav1.wav'
down_sample(man_original_data, 16000, 8000)

딥러닝을 이용하여 음성인식, 음성처리, 화자 인식 등에서 많이 쓰이는 음성 특징 추출 방법은 
1. Mel-Spectrogram
2. MFCC

#### 파라미터 별로 어떤 기능이 있는지?
1. y
2. sr
3. S :  librosa.stft(y)를 하면 얻을 수 있다. 즉 Short-Time Fourier Transform을 하여 얻어진 magnitude 와 phase의 값인 것이다. 
4. n_fft :  time-magnitude domain을 frequency로 바꾼다.
            주파수 관점에서 바라보았을 때 얻을 수 있는 정보가 많기 때문
            음성의 길이를 얼마만큼 자를지 정할 수 있다. 이를 window라 한다.
            input_nfft = int(round(sr*frame_length))
            frame_length = input_nfft/sr
5. hop_length
